In [113]:
import pandas as pd
import numpy as np
import re

#Webscraping
import requests
from requests import get
from bs4 import BeautifulSoup
import time

In [41]:
df = pd.read_csv('michelin_my_maps.csv')

In [42]:
df.head()

,Name,Address,Location,Price,Cuisine,Longitude,Latitude,PhoneNumber,Url,WebsiteUrl,Award,FacilitiesAndServices
0,King's Joy,"2 Wudaoying Hutong, Beijing, China Mainland","Beijing, China Mainland",¥¥¥,Vegetarian,116.410004,39.946681,8.610840e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,American Express credit card,..."
1,Xin Rong Ji (Xinyuan South Road),"1F, East Tower, Genesis Beijing, 8 Xinyuan Sou...","Beijing, China Mainland",¥¥¥¥,Taizhou,116.450148,39.946380,8.610650e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,Car park,China UnionPay,Count..."
2,Taian Table,"101-102, Building No. 1, Garden Office, No.161...","Shanghai, China Mainland",¥¥¥¥,Innovative,121.474049,31.221807,8.617302e+12,https://guide.michelin.com/en/shanghai-municip...,https://taian-table.cn/,3 MICHELIN Stars,"Air conditioning,American Express credit card,..."
3,Ultraviolet by Paul Pairet,'somewhere in Shanghai' - meet at Mr & Mrs Bun...,"Shanghai, China Mainland",¥¥¥¥,Innovative,121.485090,31.240358,NaN,https://guide.michelin.com/en/shanghai-municip...,https://uvbypp.cc/,3 MICHELIN Stars,"Air conditioning,American Express credit card,..."
4,Quince,"470 Pacific Ave., San Francisco, 94133, USA","San Francisco, USA",$$$$,"Contemporary, Californian",-122.403261,37.797505,1.415776e+10,https://guide.michelin.com/en/california/san-f...,http://www.quincerestaurant.com,3 MICHELIN Stars,"Air conditioning,American Express credit card,..."


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6780 entries, 0 to 6779
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Name                   6780 non-null   object 
 1   Address                6780 non-null   object 
 2   Location               6780 non-null   object 
 3   Price                  6726 non-null   object 
 4   Cuisine                6780 non-null   object 
 5   Longitude              6780 non-null   float64
 6   Latitude               6780 non-null   float64
 7   PhoneNumber            6649 non-null   float64
 8   Url                    6780 non-null   object 
 9   WebsiteUrl             5542 non-null   object 
 10  Award                  6780 non-null   object 
 11  FacilitiesAndServices  6732 non-null   object 
dtypes: float64(3), object(9)
memory usage: 635.8+ KB


In [44]:
df.Award.value_counts()

Bib Gourmand        3411
1 MICHELIN Star     2744
2 MICHELIN Stars     483
3 MICHELIN Stars     142
Name: Award, dtype: int64

In [45]:
df.Url[2]

'https://guide.michelin.com/en/shanghai-municipality/shanghai/restaurant/taian-table'

In [50]:
#Make all column names lowercase
df.columns = df.columns.str.lower()

In [46]:
#Format column names with snake case


In [51]:
#Calling restauraunt by name -- Using conventional capitalization
df[df.name == 'Rooster and the Till']

,name,address,location,price,cuisine,longitude,latitude,phonenumber,url,websiteurl,award,facilitiesandservices
5409,Rooster and the Till,"6500 N. Florida Ave., Tampa, 33604, USA","Tampa, USA",$$,International,-82.459512,28.006834,1.813375e+10,https://guide.michelin.com/en/florida/tampa/re...,https://www.roosterandthetill.com/,Bib Gourmand,"Air conditioning,American Express credit card,..."


In [53]:
#renaming columns in snake case
df = df.rename(columns = {"phonenumber":"phone_number", "websiteurl":"website_url", "facilitiesandservices":"facilities_and_services"})

In [54]:
df.head(2)

,name,address,location,price,cuisine,longitude,latitude,phone_number,url,website_url,award,facilities_and_services
0,King's Joy,"2 Wudaoying Hutong, Beijing, China Mainland","Beijing, China Mainland",¥¥¥,Vegetarian,116.410004,39.946681,8.610840e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,American Express credit card,..."
1,Xin Rong Ji (Xinyuan South Road),"1F, East Tower, Genesis Beijing, 8 Xinyuan Sou...","Beijing, China Mainland",¥¥¥¥,Taizhou,116.450148,39.946380,8.610650e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,Car park,China UnionPay,Count..."


In [55]:
df.url[0]

'https://guide.michelin.com/en/beijing-municipality/beijing/restaurant/king-s-joy'

In [56]:
df.url[1]

'https://guide.michelin.com/en/beijing-municipality/beijing/restaurant/xin-rong-ji-xinyuan-south-road'

In [57]:
url_0 = 'https://guide.michelin.com/en/beijing-municipality/beijing/restaurant/king-s-joy'

In [58]:
parts = url_0.split("en/")
parts[1]

'beijing-municipality/beijing/restaurant/king-s-joy'

In [59]:
df = df.assign(parts= df.url.str.split("en/").str[1])

In [60]:
df.head()

,name,address,location,price,cuisine,longitude,latitude,phone_number,url,website_url,award,facilities_and_services,parts
0,King's Joy,"2 Wudaoying Hutong, Beijing, China Mainland","Beijing, China Mainland",¥¥¥,Vegetarian,116.410004,39.946681,8.610840e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",beijing-municipality/beijing/restaurant/king-s...
1,Xin Rong Ji (Xinyuan South Road),"1F, East Tower, Genesis Beijing, 8 Xinyuan Sou...","Beijing, China Mainland",¥¥¥¥,Taizhou,116.450148,39.946380,8.610650e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,Car park,China UnionPay,Count...",beijing-municipality/beijing/restaurant/xin-ro...
2,Taian Table,"101-102, Building No. 1, Garden Office, No.161...","Shanghai, China Mainland",¥¥¥¥,Innovative,121.474049,31.221807,8.617302e+12,https://guide.michelin.com/en/shanghai-municip...,https://taian-table.cn/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",shanghai-municipality/shanghai/restaurant/taia...
3,Ultraviolet by Paul Pairet,'somewhere in Shanghai' - meet at Mr & Mrs Bun...,"Shanghai, China Mainland",¥¥¥¥,Innovative,121.485090,31.240358,NaN,https://guide.michelin.com/en/shanghai-municip...,https://uvbypp.cc/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",shanghai-municipality/shanghai/restaurant/ultr...
4,Quince,"470 Pacific Ave., San Francisco, 94133, USA","San Francisco, USA",$$$$,"Contemporary, Californian",-122.403261,37.797505,1.415776e+10,https://guide.michelin.com/en/california/san-f...,http://www.quincerestaurant.com,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",california/san-francisco/restaurant/quince


In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6780 entries, 0 to 6779
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   name                     6780 non-null   object 
 1   address                  6780 non-null   object 
 2   location                 6780 non-null   object 
 3   price                    6726 non-null   object 
 4   cuisine                  6780 non-null   object 
 5   longitude                6780 non-null   float64
 6   latitude                 6780 non-null   float64
 7   phone_number             6649 non-null   float64
 8   url                      6780 non-null   object 
 9   website_url              5542 non-null   object 
 10  award                    6780 non-null   object 
 11  facilities_and_services  6732 non-null   object 
 12  parts                    6780 non-null   object 
dtypes: float64(3), object(10)
memory usage: 688.7+ KB


In [47]:
urls = list(df.parts)

In [62]:
urls

NameError: name 'urls' is not defined

In [21]:
url = 'https://guide.michelin.com/en/shanghai-municipality/shanghai/restaurant/taian-table'

In [22]:
requests.get(url)
response = requests.get(url)
response

<Response [200]>

In [23]:
#telling it you will feed it html
soup = BeautifulSoup(response.content, 'html.parser')

In [30]:
soup.find_all('p', class_='restaurant-details__likeinfo--description pl-text js-favorite__description')

[<p class="restaurant-details__likeinfo--description pl-text js-favorite__description">
             Add to favorites
         </p>,
 <p class="restaurant-details__likeinfo--description pl-text js-favorite__description">
             Add to favorites
         </p>]

In [33]:
soup.find_all('p')

[<p class="restaurant-details__likeinfo--description pl-text js-favorite__description">
             Add to favorites
         </p>,
 <p class="restaurant-details__likeinfo--description pl-text js-favorite__description">
             Add to favorites
         </p>,
 <p>A fixture on the city’s dining scene since 2016, this brainchild of Chef Stefan Stiller is familiar to all local foodies. With counter seats enclosing an island where the chefs cook, guest gets to witness the birth of each course up close. The 10- or 12-course menu features 8 dishes that change entirely every 6-8 weeks. Diners then choose the rest from the classics or specials menu. The food is well-executed, sophisticated and original.</p>,
 <p>"We source from sustainable producers and minimize plastic use, food waste and energy consumption. We also recycle whenever possible and urge vendors to reduce packaging."</p>]

In [34]:
soup.select('p:not([class])')

[<p>A fixture on the city’s dining scene since 2016, this brainchild of Chef Stefan Stiller is familiar to all local foodies. With counter seats enclosing an island where the chefs cook, guest gets to witness the birth of each course up close. The 10- or 12-course menu features 8 dishes that change entirely every 6-8 weeks. Diners then choose the rest from the classics or specials menu. The food is well-executed, sophisticated and original.</p>,
 <p>"We source from sustainable producers and minimize plastic use, food waste and energy consumption. We also recycle whenever possible and urge vendors to reduce packaging."</p>]

In [75]:
soup.find('p', {'class':None})

<p>A fixture on the city’s dining scene since 2016, this brainchild of Chef Stefan Stiller is familiar to all local foodies. With counter seats enclosing an island where the chefs cook, guest gets to witness the birth of each course up close. The 10- or 12-course menu features 8 dishes that change entirely every 6-8 weeks. Diners then choose the rest from the classics or specials menu. The food is well-executed, sophisticated and original.</p>

In [76]:
soup.find('p', {'class':None}).text.strip()

'A fixture on the city’s dining scene since 2016, this brainchild of Chef Stefan Stiller is familiar to all local foodies. With counter seats enclosing an island where the chefs cook, guest gets to witness the birth of each course up close. The 10- or 12-course menu features 8 dishes that change entirely every 6-8 weeks. Diners then choose the rest from the classics or specials menu. The food is well-executed, sophisticated and original.'

In [66]:
soup.find('p', {'class':None}).contents[0]

'A fixture on the city’s dining scene since 2016, this brainchild of Chef Stefan Stiller is familiar to all local foodies. With counter seats enclosing an island where the chefs cook, guest gets to witness the birth of each course up close. The 10- or 12-course menu features 8 dishes that change entirely every 6-8 weeks. Diners then choose the rest from the classics or specials menu. The food is well-executed, sophisticated and original.'

In [77]:
df['url']

0       https://guide.michelin.com/en/beijing-municipa...
1       https://guide.michelin.com/en/beijing-municipa...
2       https://guide.michelin.com/en/shanghai-municip...
3       https://guide.michelin.com/en/shanghai-municip...
4       https://guide.michelin.com/en/california/san-f...
                              ...                        
6775    https://guide.michelin.com/en/masovia/warsaw/r...
6776    https://guide.michelin.com/en/lesser-poland/cr...
6777    https://guide.michelin.com/en/lesser-poland/cr...
6778    https://guide.michelin.com/en/masovia/warsaw/r...
6779    https://guide.michelin.com/en/masovia/warsaw/r...
Name: url, Length: 6780, dtype: object

In [79]:
def get_michelin_pages(df):
    urls = df['url']
    output = []
    for page in urls:
        soup = BeautifulSoup(requests.get(page).content)
        page_output = {'content': soup.find('p', {'class':None}).text.strip()}
        output.append(page_output)
        time.sleep(2)
    df['data'] = output
        
    return df

In [80]:
get_michelin_pages(df)

,name,address,location,price,cuisine,longitude,latitude,phone_number,url,website_url,award,facilities_and_services,parts,data
0,King's Joy,"2 Wudaoying Hutong, Beijing, China Mainland","Beijing, China Mainland",¥¥¥,Vegetarian,116.410004,39.946681,8.610840e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",beijing-municipality/beijing/restaurant/king-s...,{'page': 'https://guide.michelin.com/en/beijin...
1,Xin Rong Ji (Xinyuan South Road),"1F, East Tower, Genesis Beijing, 8 Xinyuan Sou...","Beijing, China Mainland",¥¥¥¥,Taizhou,116.450148,39.946380,8.610650e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,Car park,China UnionPay,Count...",beijing-municipality/beijing/restaurant/xin-ro...,{'page': 'https://guide.michelin.com/en/beijin...
2,Taian Table,"101-102, Building No. 1, Garden Office, No.161...","Shanghai, China Mainland",¥¥¥¥,Innovative,121.474049,31.221807,8.617302e+12,https://guide.michelin.com/en/shanghai-municip...,https://taian-table.cn/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",shanghai-municipality/shanghai/restaurant/taia...,{'page': 'https://guide.michelin.com/en/shangh...
3,Ultraviolet by Paul Pairet,'somewhere in Shanghai' - meet at Mr & Mrs Bun...,"Shanghai, China Mainland",¥¥¥¥,Innovative,121.485090,31.240358,NaN,https://guide.michelin.com/en/shanghai-municip...,https://uvbypp.cc/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",shanghai-municipality/shanghai/restaurant/ultr...,{'page': 'https://guide.michelin.com/en/shangh...
4,Quince,"470 Pacific Ave., San Francisco, 94133, USA","San Francisco, USA",$$$$,"Contemporary, Californian",-122.403261,37.797505,1.415776e+10,https://guide.michelin.com/en/california/san-f...,http://www.quincerestaurant.com,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",california/san-francisco/restaurant/quince,{'page': 'https://guide.michelin.com/en/califo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6775,Szóstka,"Plac Powstańców Warszawy 9, Warsaw, 00 039, Po...","Warsaw, Poland",€€,"Polish, Modern Cuisine",21.012698,52.235803,4.822470e+10,https://guide.michelin.com/en/masovia/warsaw/r...,https://warszawa.hotel.com.pl/hotel-warszawa,Bib Gourmand,"Air conditioning,American Express credit card,...",masovia/warsaw/restaurant/szostka,{'page': 'https://guide.michelin.com/en/masovi...
6776,Fiorentina,"ul. Grodzka 63, Cracow, 31 044, Poland","Cracow, Poland",€€,"Creative, Polish",19.938179,50.055898,4.812426e+10,https://guide.michelin.com/en/lesser-poland/cr...,https://fiorentina.com.pl/,Bib Gourmand,"Air conditioning,American Express credit card,...",lesser-poland/cracow/restaurant/fiorentina,{'page': 'https://guide.michelin.com/en/lesser...
6777,Zazie,"ul. Józefa 34, Cracow, 32 056, Poland","Cracow, Poland",€,French,19.946949,50.051240,4.850041e+10,https://guide.michelin.com/en/lesser-poland/cr...,http://www.zaziebistro.pl/,Bib Gourmand,"Air conditioning,Booking essential - dinner,Cr...",lesser-poland/cracow/restaurant/zazie,{'page': 'https://guide.michelin.com/en/lesser...
6778,Butchery & Wine,"ul. Żurawia 22, Warsaw, 00 515, Poland","Warsaw, Poland",€€,"Meats and Grills, Traditional Cuisine",21.015495,52.228581,4.822502e+10,https://guide.michelin.com/en/masovia/warsaw/r...,https://www.butcheryandwine.pl/,Bib Gourmand,"American Express credit card,Booking essential...",masovia/warsaw/restaurant/butchery-and-wine,{'page': 'https://guide.michelin.com/en/masovi...


In [86]:
df.data[6779]

{'page': 'https://guide.michelin.com/en/masovia/warsaw/restaurant/alewino',
 'content': 'Alewino started life as a wine shop before developing into a wine-bar-cum-restaurant with 4 cosy rooms. The menus might be concise but portions are generous, with classic Polish recipes reworked in a modern manner. Over 250 great value wines are available by the glass; ask for recommendations.'}

In [105]:
example_string = df.data[6779]

In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6780 entries, 0 to 6779
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   name                     6780 non-null   object 
 1   address                  6780 non-null   object 
 2   location                 6780 non-null   object 
 3   price                    6726 non-null   object 
 4   cuisine                  6780 non-null   object 
 5   longitude                6780 non-null   float64
 6   latitude                 6780 non-null   float64
 7   phone_number             6649 non-null   float64
 8   url                      6780 non-null   object 
 9   website_url              5542 non-null   object 
 10  award                    6780 non-null   object 
 11  facilities_and_services  6732 non-null   object 
 12  parts                    6780 non-null   object 
 13  data                     6780 non-null   object 
dtypes: float64(3), object(11

In [98]:
df['data'].str.extract(',:}')

ValueError: pattern contains no capture groups

In [106]:
example_string

{'page': 'https://guide.michelin.com/en/masovia/warsaw/restaurant/alewino',
 'content': 'Alewino started life as a wine shop before developing into a wine-bar-cum-restaurant with 4 cosy rooms. The menus might be concise but portions are generous, with classic Polish recipes reworked in a modern manner. Over 250 great value wines are available by the glass; ask for recommendations.'}

In [112]:
df['data']strip("{}")

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
6775   NaN
6776   NaN
6777   NaN
6778   NaN
6779   NaN
Name: data, Length: 6780, dtype: float64

In [114]:
example_string = re.sub(r'[{}]', '', example_string)

TypeError: expected string or bytes-like object